# Set up 
Load the required packages. 

In [6]:
# set up 
import pandas as pd 
import numpy as np 
import math 
import os 
from os.path import join as joinpath 
import downstream 
import upstream 

# Import raw data 
Import the raw data of Consumer Price Index(CPI), interest rate of 1-year national bond, and settlement price of commodity futures. 

In [4]:
# import raw data 
cpi_raw = downstream.get_cpi_raw() 
bond_raw = downstream.get_bond_raw() 
future_raw = downstream.get_future_raw() 

# Data processing 
## Inflation rate 
The raw CPI is monthly year-on-year data, and then take logarithm form as inflation rate. 

The national bond is daily data, so here takes the data of last day in each months as monthly data, and then take 1-month lagged value as expected inflation rate. 

In [5]:
# inflation rate 
cpi_raw['date'] = pd.to_datetime(cpi_raw['date'], infer_datetime_format = True) 
cpi_raw['inflation_rate'] = cpi_raw['CPI'].apply(np.log) 
cpi_raw.sort_values(by = 'date', inplace = True) 

print(cpi_raw.dtypes) 
cpi_raw.head() 

date              datetime64[ns]
CPI                      float64
inflation_rate           float64
dtype: object


,date,CPI,inflation_rate
71,2010-01-01,101.5,4.620059
70,2010-02-01,102.7,4.631812
69,2010-03-01,102.4,4.628887
68,2010-04-01,102.8,4.632785
67,2010-05-01,103.1,4.635699


In [7]:
# national bond (expected inflation rate)
bond_raw['date'] = pd.to_datetime(bond_raw['date'], infer_datetime_format = True) 
bond_raw['month'] = bond_raw['date'].apply(lambda x: pd.to_datetime(str(x)[:7] + '-01', infer_datetime_format = True)) 

print(bond_raw.dtypes) 

bond_month = bond_raw.groupby('month').apply(lambda t: t[t.date == t.date.max()]) 
bond_month = pd.DataFrame(bond_month[['month', '1_year']])
bond_month.reset_index(drop = True, inplace = True) 
bond_month['expected_inflation_rate'] = bond_month['1_year'].shift(1) 

print(bond_month.dtypes) 
bond_month.head() 

date      datetime64[ns]
1_year           float64
month     datetime64[ns]
dtype: object
month                      datetime64[ns]
1_year                            float64
expected_inflation_rate           float64
dtype: object


,month,1_year,expected_inflation_rate
0,2009-12-01,1.4953,NaN
1,2010-01-01,1.6529,1.4953
2,2010-02-01,1.6294,1.6529
3,2010-03-01,1.5501,1.6294
4,2010-04-01,1.5892,1.5501


In [8]:
# merge inflation data 
inflation = pd.merge(cpi_raw[['date', 'CPI', 'inflation_rate']], bond_month[['month', '1_year', 'expected_inflation_rate']], how = 'outer', left_on = 'date', right_on = 'month') 
inflation = inflation[['date', 'CPI', 'inflation_rate', '1_year', 'expected_inflation_rate']]
inflation.sort_values(by = 'date', inplace = True) 

print(inflation.dtypes) 
inflation.head() 

date                       datetime64[ns]
CPI                               float64
inflation_rate                    float64
1_year                            float64
expected_inflation_rate           float64
dtype: object


,date,CPI,inflation_rate,1_year,expected_inflation_rate
0,2010-01-01,101.5,4.620059,1.6529,1.4953
1,2010-02-01,102.7,4.631812,1.6294,1.6529
2,2010-03-01,102.4,4.628887,1.5501,1.6294
3,2010-04-01,102.8,4.632785,1.5892,1.5501
4,2010-05-01,103.1,4.635699,1.8162,1.5892


## Commodity futures 
The return of commodity futures, which is the logarithm of one-year holding period return, is computed with the settlement price, named as the market volume weighted strike price of the contract, at the end of each month. 

In [9]:
# future return 
future_raw['date'] = pd.to_datetime(future_raw['date'], infer_datetime_format = True) 
future_raw.sort_values(by = 'date', inplace = True) 

print(future_raw.dtypes) 

col_names = future_raw.columns[1 : ] 
future_log_ret = future_raw['date']
for col in col_names: 
  future_one = future_raw[['date', col]] 
  future_one['lag'] = future_one[col].shift(12) 
  future_one[col + '_ret'] = future_one[col].apply(math.log) - future_one['lag'].apply(math.log) 
  future_log_ret = pd.merge(future_log_ret, future_one[['date', col + '_ret']], how = 'left', on = 'date')

print(future_log_ret.dtypes) 
future_log_ret.head() 

date              datetime64[ns]
Aluminum                 float64
Cathode_Copper           float64
Fuel_Oil                 float64
Gold                     float64
LLDPE                    float64
Natural_Rubber           float64
Palm_Oil                 float64
PVC                      float64
Rebar                    float64
Soybean_Meal             float64
Soybean_Oil              float64
Soybeans_No_1            float64
Soybeans_No_2            float64
Wire_Rod                 float64
Yellow_Corn              float64
Zinc                     float64
dtype: object
date                  datetime64[ns]
Aluminum_ret                 float64
Cathode_Copper_ret           float64
Fuel_Oil_ret                 float64
Gold_ret                     float64
LLDPE_ret                    float64
Natural_Rubber_ret           float64
Palm_Oil_ret                 float64
PVC_ret                      float64
Rebar_ret                    float64
Soybean_Meal_ret             float64
Soybean_Oil_ret   

/var/folders/kj/6m4r1_gn0g13yqhqd0ythj7w0000gn/T/ipykernel_90566/2101770634.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_one['lag'] = future_one[col].shift(12)
/var/folders/kj/6m4r1_gn0g13yqhqd0ythj7w0000gn/T/ipykernel_90566/2101770634.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_one[col + '_ret'] = future_one[col].apply(math.log) - future_one['lag'].apply(math.log)
/var/folders/kj/6m4r1_gn0g13yqhqd0ythj7w0000gn/T/ipykernel_90566/2101770634.py:11: SettingWithCopyWarning: 
A val

,date,Aluminum_ret,Cathode_Copper_ret,Fuel_Oil_ret,Gold_ret,LLDPE_ret,Natural_Rubber_ret,Palm_Oil_ret,PVC_ret,Rebar_ret,Soybean_Meal_ret,Soybean_Oil_ret,Soybeans_No_1_ret,Soybeans_No_2_ret,Wire_Rod_ret,Yellow_Corn_ret,Zinc_ret
0,2009-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge data for modelling 
Merge all the monthly returns for modelling. 

In [10]:
# merge data 
model_data = pd.merge(inflation[['date', 'inflation_rate', 'expected_inflation_rate']], future_log_ret, how = 'outer', on = 'date')

# select date 
model_data = model_data.loc[(model_data['date'] >= '2010-01-01') & (model_data['date'] <= '2014-03-01')]

# export data 
upstream.put_model_data_pro(model_data) 

print(model_data.dtypes) 
model_data.head() 

date                       datetime64[ns]
inflation_rate                    float64
expected_inflation_rate           float64
Aluminum_ret                      float64
Cathode_Copper_ret                float64
Fuel_Oil_ret                      float64
Gold_ret                          float64
LLDPE_ret                         float64
Natural_Rubber_ret                float64
Palm_Oil_ret                      float64
PVC_ret                           float64
Rebar_ret                         float64
Soybean_Meal_ret                  float64
Soybean_Oil_ret                   float64
Soybeans_No_1_ret                 float64
Soybeans_No_2_ret                 float64
Wire_Rod_ret                      float64
Yellow_Corn_ret                   float64
Zinc_ret                          float64
dtype: object


,date,inflation_rate,expected_inflation_rate,Aluminum_ret,Cathode_Copper_ret,Fuel_Oil_ret,Gold_ret,LLDPE_ret,Natural_Rubber_ret,Palm_Oil_ret,PVC_ret,Rebar_ret,Soybean_Meal_ret,Soybean_Oil_ret,Soybeans_No_1_ret,Soybeans_No_2_ret,Wire_Rod_ret,Yellow_Corn_ret,Zinc_ret
0,2010-01-01,4.620059,1.4953,0.390941,0.802988,0.471230,0.303510,0.391057,0.686334,0.262608,NaN,NaN,0.110199,0.177870,0.151676,0.181882,NaN,0.190759,0.634747
1,2010-02-01,4.631812,1.6529,0.332852,0.694743,0.377658,0.160871,0.300203,0.585697,0.246007,NaN,NaN,0.045205,0.167340,0.090586,0.116789,NaN,0.118553,0.541737
2,2010-03-01,4.628887,1.6294,0.302778,0.655327,0.397492,0.190301,0.291501,0.616913,0.264161,NaN,0.269494,0.074168,0.191274,0.099368,0.121242,0.214797,0.113616,0.532738
3,2010-04-01,4.632785,1.5501,0.265700,0.514376,0.341045,0.251650,0.222391,0.525593,0.122120,NaN,0.296044,0.023808,0.131463,0.120716,0.127206,0.273467,0.136503,0.422531
4,2010-05-01,4.635699,1.5892,0.188014,0.399383,0.228898,0.267881,0.097995,0.379759,-0.008189,0.141971,0.184393,0.006001,0.010433,0.093578,0.083921,0.182693,0.154434,0.229693
